In [10]:
import requests
from bs4 import BeautifulSoup
from transformers import pipeline
import os
from urllib.parse import urlparse, unquote
from transformers import LEDForConditionalGeneration, LEDTokenizer

def get_article_text(url):
    response = requests.get(url)
    encoding = response.encoding if 'charset' in response.headers.get('content-type', '').lower() else 'utf-8'
    decoded_content = response.content.decode(encoding)
    soup = BeautifulSoup(decoded_content, 'html.parser')
    title = soup.find('title').text
    paragraphs = soup.find_all('p')
    article_text = '\n'.join([p.get_text() for p in paragraphs])
    return title, article_text


def summarize_text(text, max_length=500):
    summarizer = pipeline("summarization")
    summary = summarizer(text, max_length=max_length, do_sample=False, clean_up_tokenization_spaces=True)
    return summary[0]['summary_text']

def summarize_article(url):
    article_text = get_article_text(url)
    summarized_text = summarize_text(article_text)
    return summarized_text

def write_summarized_to_file(summarized_text, url):
    # Parse the URL and extract the path
    parsed_url = urlparse(url)
    path = parsed_url.path

    # Remove the leading/trailing slashes and split the path by slashes
    path_parts = path.strip('/').split('/')

    # Get the last part of the path and decode any URL-encoded characters
    last_part = unquote(path_parts[-1])

    # Create the filename with a .txt extension
    filename = f"{last_part}.md"

    # Write the summarized text to the file
    with open(filename, "w") as file:
        file.write(summarized_text)

    print(f"Summarized text saved to {filename}")



In [11]:
def summarize_text(text, max_length=256):
    model_name = "facebook/bart-large-cnn"
    tokenizer = BartTokenizer.from_pretrained(model_name)
    model = BartForConditionalGeneration.from_pretrained(model_name)

    input_ids = tokenizer(text, return_tensors="pt", padding="max_length", truncation=True, max_length=1024).input_ids
    summary_ids = model.generate(input_ids, num_beams=4, length_penalty=2.0, max_length=max_length, min_length=50)
    summary_text = tokenizer.decode(summary_ids[0], skip_special_tokens=True)

    return summary_text

In [ ]:
url = "https://www.engadget.com/chatgpts-new-plugins-will-deliver-real-time-stats-182900388.html"
summarized_text = summarize_article(url)
print(summarized_text)
write_summarized_to_file(summarized_text=summarized_text, url=url)

In [ ]:
url = "https://newsroom.churchofjesuschrist.org/article/temple-news-from-north-and-central-america"
summarized_text = summarize_article(url)
print(summarized_text)
write_summarized_to_file(summarized_text=summarized_text, url=url)

No model was supplied, defaulted to sshleifer/distilbart-cnn-12-6 and revision a4f8f3e (https://huggingface.co/sshleifer/distilbart-cnn-12-6).
Using a pipeline without specifying a model name and revision in production is not recommended.


 The First Presidency of The Church of Jesus Christ of Latter-day Saints has released the open house and dedication dates for the Feather River California Temple. In addition, the location of the Huehuetenango Guatemala Temple and a rendering of the Tampa Florida Temple have also been announced. There are 10 temples in California, the most of any state besides Utah.


In [5]:
url = "https://newsroom.churchofjesuschrist.org/article/2022-annual-report-caring-for-those-in-need"
summarized_text = summarize_article(url)
print(summarized_text)
write_summarized_to_file(summarized_text=summarized_text, url=url)

No model was supplied, defaulted to sshleifer/distilbart-cnn-12-6 and revision a4f8f3e (https://huggingface.co/sshleifer/distilbart-cnn-12-6).
Using a pipeline without specifying a model name and revision in production is not recommended.
Token indices sequence length is longer than the specified maximum sequence length for this model (1318 > 1024). Running this sequence through the model will result in indexing errors


IndexError: index out of range in self

In [16]:
from IPython.display import Markdown as md
url = "https://www.espn.com/nba/story/_/id/35948701/hawks-trae-young-ejected-firing-ball-official"
display(md(get_article_text(url)[0]))


Hawks' Trae Young ejected after firing ball at official